In [4]:
import scipy.io as sio
import torch
import numpy as np
import os
import random
import torch.optim as optim
from torch import Tensor
import torch.nn as nn
from DenseNet import creat_densenet
from se_densenet import creat_sedensenet
#import winsound

In [5]:
class GetLoader(torch.utils.data.Dataset):
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels
    def __len__(self):
        return len(self.data)

In [9]:
local_dir='/gpfs/share/home/2000012146/testing38/'
dirnames=os.listdir(local_dir)

In [10]:
train_data=[]
train_label=[]
test_data=[]
test_label=[]

In [11]:
for Dirname in dirnames:
    if Dirname=='10000_cubic_50_4_0_10_null_250_10-14':
        l=0
    elif Dirname=='10000_cubic_50_4_10max_10_10max_250_10-14':
        l=1
    filenames=os.listdir(local_dir+Dirname+'/')
    i=0
    for Filename in filenames:
        filename=local_dir+Dirname+'/'+Filename
        if i<8000:
            train_data.append(np.load(filename)[0:128,0:128].astype(np.float32).T.reshape(1,128,128))
            train_label.append(l)
        else:
            test_data.append(np.load(filename)[0:128,0:128].astype(np.float32).T.reshape(1,128,128))
            test_label.append(l)
        i+=1

In [12]:
train_label=np.array(train_label)
test_label=np.array(test_label)
train_Label_data = GetLoader(train_data, train_label)
Train_dataload= torch.utils.data.DataLoader(train_Label_data, batch_size=3, shuffle=True, drop_last=False)
test_Label_data=GetLoader(test_data, test_label)
Test_dataload= torch.utils.data.DataLoader(test_Label_data, batch_size=3, shuffle=True, drop_last=False)

In [13]:
#writer = SummaryWriter()
classes = tuple(dirnames)

net = creat_sedensenet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
 
device = torch.device("cuda:0")
net.to(device)

SEDenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (norm0): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (SELayer_0a): SELayer(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (fc): Sequential(
        (0): Linear(in_features=24, out_features=1, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=1, out_features=24, bias=True)
        (3): Sigmoid()
      )
    )
    (SELayer_1a): SELayer(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (fc): Sequential(
        (0): Linear(in_features=24, out_features=1, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=1, out_features=24, bias=True)
        (3): Sigmoid()
      )
    )
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (selayer): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
    

In [ ]:
for epoch in range(30):
    
    for i, data in enumerate(Train_dataload):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)

        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()

    pklname=("se_epoch_%d.pkl"%epoch)     
    torch.save(net,pklname)
    correct = 0
    total = 0
    CM=np.zeros((3,3))
    with torch.no_grad():
        for data in Test_dataload:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            labels=labels.long()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            for pre,lab in zip(predicted,labels):
                CM[pre,lab]+=1
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            
    print('Accuracy of se_%d: %d %%' % (epoch,100 * correct / total))

    for j in range(2):
        CM[j,2]=sum(CM[j,0:2])
    for j in range(3):
        CM[2,j]=sum(CM[0:2,j])
    print(CM)
    #writer.add_scalar('accuracy', correct / total, epoch+1)
    np.save('se_CM_epoch%d.npy'%epoch,CM)
    #winsound.Beep(1000,1000)


print('Finished Training')

Accuracy of se_0: 61 %
[[1878. 1418. 3296.]
 [ 122.  582.  704.]
 [2000. 2000. 4000.]]
Accuracy of se_1: 66 %
[[1854. 1188. 3042.]
 [ 146.  812.  958.]
 [2000. 2000. 4000.]]
Accuracy of se_2: 67 %
[[1835. 1139. 2974.]
 [ 165.  861. 1026.]
 [2000. 2000. 4000.]]
Accuracy of se_3: 67 %
[[1736. 1026. 2762.]
 [ 264.  974. 1238.]
 [2000. 2000. 4000.]]
Accuracy of se_4: 67 %
[[1477.  794. 2271.]
 [ 523. 1206. 1729.]
 [2000. 2000. 4000.]]
Accuracy of se_5: 66 %
[[1435.  784. 2219.]
 [ 565. 1216. 1781.]
 [2000. 2000. 4000.]]
Accuracy of se_6: 68 %
[[1727.  991. 2718.]
 [ 273. 1009. 1282.]
 [2000. 2000. 4000.]]
Accuracy of se_7: 67 %
[[1489.  788. 2277.]
 [ 511. 1212. 1723.]
 [2000. 2000. 4000.]]
Accuracy of se_8: 70 %
[[1685.  866. 2551.]
 [ 315. 1134. 1449.]
 [2000. 2000. 4000.]]
Accuracy of se_9: 71 %
[[1505.  655. 2160.]
 [ 495. 1345. 1840.]
 [2000. 2000. 4000.]]
Accuracy of se_10: 71 %
[[1739.  880. 2619.]
 [ 261. 1120. 1381.]
 [2000. 2000. 4000.]]
Accuracy of se_11: 69 %
[[1199.  407. 1606